In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 


pybullet build time: Jan 29 2025 23:19:57


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3050 Ti Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.230.02
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.230.02
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3050 Ti Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation


In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")



ven = NVIDIA Corporation


In [3]:
# Load the robot
car_start_pos = [0, 0, 0.2]  # Set the initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/mobile_robot_with_arm.urdf", car_start_pos, car_start_orientation)

In [4]:
# Get joint indices for the robot arm and gripper
# UR5 arm joint names
arm_joint_names = [
    'shoulder_pan_joint',
    'shoulder_lift_joint',
    'elbow_joint',
    'wrist_1_joint',
    'wrist_2_joint',
    'wrist_3_joint'
]

# Gripper joint names (both need to be controlled since PyBullet doesn't support mimic joints)
gripper_right_joint_name = 'robotiq_2f_85_right_driver_joint'
gripper_left_joint_name = 'robotiq_2f_85_left_driver_joint'

# Function to get joint index by name
def get_joint_index_by_name(robot_id, joint_name):
    """Get the joint index by searching through all joints."""
    num_joints = pybullet.getNumJoints(robot_id)
    for i in range(num_joints):
        joint_info = pybullet.getJointInfo(robot_id, i)
        if joint_info[1].decode('utf-8') == joint_name:
            return i
    return None

# Get joint indices for arm and gripper
arm_joint_indices = {}
for joint_name in arm_joint_names:
    joint_idx = get_joint_index_by_name(car_id, joint_name)
    arm_joint_indices[joint_name] = joint_idx
    if joint_idx is not None:
        print(f"{joint_name} index: {joint_idx}")
    else:
        print(f"Warning: {joint_name} not found!")

# Get gripper joint indices (both right and left)
gripper_right_joint_idx = get_joint_index_by_name(car_id, gripper_right_joint_name)
gripper_left_joint_idx = get_joint_index_by_name(car_id, gripper_left_joint_name)

if gripper_right_joint_idx is not None:
    print(f"{gripper_right_joint_name} index: {gripper_right_joint_idx}")
else:
    print(f"Warning: {gripper_right_joint_name} not found!")

if gripper_left_joint_idx is not None:
    print(f"{gripper_left_joint_name} index: {gripper_left_joint_idx}")
else:
    print(f"Warning: {gripper_left_joint_name} not found!")


shoulder_pan_joint index: 12
shoulder_lift_joint index: 13
elbow_joint index: 14
wrist_1_joint index: 15
wrist_2_joint index: 16
wrist_3_joint index: 17
robotiq_2f_85_right_driver_joint index: 20
robotiq_2f_85_left_driver_joint index: 25


In [ ]:
# Define joint limits (from URDF)
joint_limits = {
    'shoulder_pan_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'shoulder_lift_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'elbow_joint': (-3.14159265359, 3.14159265359),  # ±180°
    'wrist_1_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'wrist_2_joint': (-6.28318530718, 6.28318530718),  # ±360°
    'wrist_3_joint': (-6.28318530718, 6.28318530718)   # ±360°
}

# Gripper limits (0.0 = closed, 0.834 = fully open)
gripper_limits = (0.0, 0.834)

# Joint effort limits (maximum force/torque for each joint)
joint_efforts = {
    'shoulder_pan_joint': 150.0,
    'shoulder_lift_joint': 150.0,
    'elbow_joint': 150.0,
    'wrist_1_joint': 28.0,
    'wrist_2_joint': 28.0,
    'wrist_3_joint': 28.0
}

gripper_effort = 60.0  # Maximum force for gripper

joint_default = 0.0  # Start at 0 radians
gripper_default = 0.0  # Start closed

# Add sliders for each arm joint in PyBullet GUI
# These will appear in the right panel of the PyBullet window
slider_ids = {}
for joint_name in arm_joint_names:
    joint_min, joint_max = joint_limits[joint_name]
    slider_id = pybullet.addUserDebugParameter(
        joint_name,  # Parameter name
        joint_min,   # Minimum value (radians)
        joint_max,   # Maximum value (radians)
        joint_default  # Default value
    )
    slider_ids[joint_name] = slider_id
    print(f"Added slider for {joint_name} (slider ID: {slider_id}, range: [{joint_min:.2f}, {joint_max:.2f}] rad)")

# Add slider for gripper (single slider for opening/closing)
gripper_slider_id = pybullet.addUserDebugParameter(
    "Gripper Open/Close",  # Parameter name
    gripper_limits[0],     # Minimum value (0.0 = closed)
    gripper_limits[1],     # Maximum value (0.834 = fully open)
    gripper_default        # Default value (closed)
)
print(f"Added slider for Gripper (slider ID: {gripper_slider_id}, range: [{gripper_limits[0]:.2f}, {gripper_limits[1]:.2f}] rad)")


Added slider for shoulder_pan_joint (slider ID: 0, range: [-6.28, 6.28] rad)
Added slider for shoulder_lift_joint (slider ID: 1, range: [-6.28, 6.28] rad)
Added slider for elbow_joint (slider ID: 2, range: [-3.14, 3.14] rad)
Added slider for wrist_1_joint (slider ID: 3, range: [-6.28, 6.28] rad)
Added slider for wrist_2_joint (slider ID: 4, range: [-6.28, 6.28] rad)
Added slider for wrist_3_joint (slider ID: 5, range: [-6.28, 6.28] rad)
Added slider for Gripper (slider ID: 6, range: [0.00, 0.83] rad)


: 

In [ ]:
# Control loop: Read slider values and control robot arm and gripper
# This will run continuously until interrupted

import time

print("Starting control loop. Use the sliders in the PyBullet GUI to control the robot.")
print("Press Ctrl+C in the terminal to stop.")

try:
    while True:
        # Read slider values and control arm joints
        for joint_name in arm_joint_names:
            if joint_name in slider_ids and joint_name in arm_joint_indices:
                slider_id = slider_ids[joint_name]
                joint_idx = arm_joint_indices[joint_name]
                if joint_idx is not None:
                    # Read slider value
                    joint_angle = pybullet.readUserDebugParameter(slider_id)
                    # Get joint effort limit (force/torque)
                    joint_effort = joint_efforts.get(joint_name, 150.0)
                    # Set joint position (control mode: position control)
                    pybullet.setJointMotorControl2(
                        car_id,
                        joint_idx,
                        pybullet.POSITION_CONTROL,
                        targetPosition=joint_angle,
                        force=joint_effort  # Maximum force/torque for this joint
                    )
        
        # Control gripper (both right and left driver joints)
        # Read gripper slider value once
        gripper_angle = pybullet.readUserDebugParameter(gripper_slider_id)
        
        # Control right driver joint
        if gripper_right_joint_idx is not None:
            pybullet.setJointMotorControl2(
                car_id,
                gripper_right_joint_idx,
                pybullet.POSITION_CONTROL,
                targetPosition=gripper_angle,
                force=gripper_effort
            )
        
        # Control left driver joint (same angle as right, since they should move together)
        if gripper_left_joint_idx is not None:
            pybullet.setJointMotorControl2(
                car_id,
                gripper_left_joint_idx,
                pybullet.POSITION_CONTROL,
                targetPosition=gripper_angle,
                force=gripper_effort
            )
        
        # Step simulation
        pybullet.stepSimulation()
        
        # Small delay to prevent excessive CPU usage
        time.sleep(time_step)
        
except KeyboardInterrupt:
    print("\nControl loop stopped.")


Starting control loop. Use the sliders in the PyBullet GUI to control the robot.
Press Ctrl+C in the terminal to stop.
